In [58]:
import json
import requests
from lxml import html
from bs4 import BeautifulSoup
from collections import OrderedDict
import argparse
import re
import dateutil.parser
import pandas as pd
import time
from selenium import webdriver


In [59]:
def fix_K_or_M(n):
    if ' Follower' in str(n):
        n = re.sub(' Follower','',n)
        return int(n)
    elif 'K' in str(n):
        n = re.sub('K','',n)
        n = float(n) * 1000
        return int(n)
    elif 'M' in str(n):
        n = re.sub('M','',n)
        n = float(n) * 1000000
        return int(n)
    else: 
        n = float(n)
        return int(n)

In [60]:
def count_followers(soup):
    try: 
        followers = soup.find(attrs ={'data-action-value':'followers'})
        follower_count = followers.text
        follower_count = follower_count.replace(" Followers", "")
        follower_count = fix_K_or_M(follower_count)
        return follower_count
    except AttributeError:
        return 0
        
def count_leaders(soup):
    try: 
        following = soup.find(attrs ={'data-action-value':'following'})
        following_count= following.text
        following_count = following_count.replace(" Following", "")
        following_count = fix_K_or_M(following_count)
        return following_count
    except AttributeError:
        return 0

def get_name(soup):
    name = soup.find('h1',{'class':'ui-h2 hero-title' })
    return name.text

## Get user followers and leaders

In [61]:
#try loading page with selenium, then scrpaing
driver = webdriver.Chrome(executable_path="/Users/mayamidzik/tools/chromedriver")
url='https://medium.com/@SenateGOP'
driver.get(url)
print(driver.current_url)

https://medium.com/@SenateGOP


In [62]:
initialHTML = driver.execute_script("return document.body.innerHTML") #returns the inner HTML as a string
initialsoup = BeautifulSoup(initialHTML,"lxml")

## count initial followers & leaders

In [63]:
followers_count = count_followers(initialsoup)
leaders_count = count_leaders(initialsoup)

print('followers: '+ str(followers_count))
print('leaders: '+ str(leaders_count))

followers: 10700
leaders: 2000


In [64]:
#try to fix thousands endings (see @garyvee for example)
# followers_count= followers_count.replace(" Followers", "")
# if followers_count[-1] == K

## Get a dictionary of followers & save to pickle
* navigate to followers page
* scroll to page end (based on followers count)
* get html, find all elemts with followers & save to dicitonary\
* dave dict to pd & pickle

In [65]:
followers_url = url+ '/followers'
driver.get(followers_url)

In [205]:
# try:
#     print(int(followers_count))
# except:
#     followers_count = re.sub('K','',followers_count)
#     followers_count = int(followers_count) * 1000
#     print(followers_count)
    

2000


In [68]:
# selenium scroll to end of page based on a given numer
i = 0
dicts = {}
stream = []
catch_error = []
while len(stream) < followers_count:
    try:
        time.sleep(.4)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        stream = driver.find_elements_by_class_name('ui-captionStrong')
        catch_error.append(len(stream))
        if len(catch_error)>6:
            #this is a super janky way to test for a timeout error
            if catch_error[-2] == catch_error[-3] == catch_error[-4] == catch_error[-5] == catch_error[-6] == catch_error[-7]:
                break
                print('hung before end')
    except TimeoutException:
        pass

In [69]:
#get the soup data
innerHTML = driver.execute_script("return document.body.innerHTML")
soup = BeautifulSoup(innerHTML,"lxml")

In [70]:
#get the node's followers and create correct dict
def get_followers(soup):
    followers_list = set()
    followers = soup.find_all('a', {'class':'link u-baseColor--link avatar u-width60 u-marginRight20 u-flex0'})
    for follower in followers:
        follower = follower['href']
        followers_list.add(follower)
    return followers_list

def get_followers_dict(soup):
    followers = soup.find_all('a', {'class':'link u-baseColor--link avatar u-width60 u-marginRight20 u-flex0'})
    i = 0
    followers_dict = {}
    for follower in followers:
        i+=1
        follower = follower['href']
        #should put in the node's url here
        followers_dict[i] = [url,follower]
    return followers_dict

### keep master list of people: all_the_people = set()

In [71]:
all_the_people = set()
all_the_people.update(get_followers(soup))
all_the_people;


#add a single word
#all_the_people.add(('https://medium.com/@chad.scherrer'))

#create a set and update with multiple words
#s = {'this','that'}
#all_the_people.update(s)
#all_the_people

In [72]:
followers = get_followers_dict(soup)

In [73]:
followers

{1: ['https://medium.com/@SenateGOP', 'https://medium.com/@tannonbaum'],
 2: ['https://medium.com/@SenateGOP', 'https://medium.com/@ChandlerHudson'],
 3: ['https://medium.com/@SenateGOP', 'https://medium.com/@JosephMaynard'],
 4: ['https://medium.com/@SenateGOP', 'https://medium.com/@Richard_Zoller'],
 5: ['https://medium.com/@SenateGOP', 'https://medium.com/@fredvanhook'],
 6: ['https://medium.com/@SenateGOP', 'https://medium.com/@allisonmayer'],
 7: ['https://medium.com/@SenateGOP', 'https://medium.com/@JonneePAc'],
 8: ['https://medium.com/@SenateGOP', 'https://medium.com/@okieoilman'],
 9: ['https://medium.com/@SenateGOP', 'https://medium.com/@grizzliesrule888'],
 10: ['https://medium.com/@SenateGOP', 'https://medium.com/@JohnRisvold'],
 11: ['https://medium.com/@SenateGOP', 'https://medium.com/@k8lyman'],
 12: ['https://medium.com/@SenateGOP', 'https://medium.com/@indianz'],
 13: ['https://medium.com/@SenateGOP', 'https://medium.com/@scott_keyvon'],
 14: ['https://medium.com/@Sena

In [74]:
len(followers)

1154

In [75]:
followers_j = pd.DataFrame(followers).T
followers_j.rename(columns = {0:'leader',1:'follower'}, inplace=True)
followers_j.tail()

,leader,follower
1150,https://medium.com/@SenateGOP,https://medium.com/@ChuckFinnell
1151,https://medium.com/@SenateGOP,https://medium.com/@JoeC1776
1152,https://medium.com/@SenateGOP,https://medium.com/@barbedwireny
1153,https://medium.com/@SenateGOP,https://medium.com/@AnikaReynolds
1154,https://medium.com/@SenateGOP,https://medium.com/@JNLevin


In [76]:
followers_j.to_pickle('SenateGOP_followers.pkl')

## Get a dictionary of leaders & save to pickle
* navigate to following page
* scroll to page end (based on followers count)
* get html, find all elements with leaders & save to dicitonary\
* save dict to pd & pickle

In [77]:
leaders_url = url+ '/following'
driver.get(leaders_url)

In [78]:
print(int(leaders_count))

2000


In [79]:
# selenium scroll to end of page based on a given number
i = 0
dicts = {}
stream = []
catch_error = []
while len(stream) < int(leaders_count):
    try:
        time.sleep(.4)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        stream = driver.find_elements_by_class_name('ui-captionStrong')
        catch_error.append(len(stream))
        if len(catch_error)>6:
            #this is a super janky way to test for a timeout error
            if catch_error[-2] == catch_error[-3] == catch_error[-4] == catch_error[-5] == catch_error[-6] == catch_error[-7]:
                break
                print('hung before end')
    except TimeoutException:
        pass
            
            

In [80]:
#get the soup data
innerHTML = driver.execute_script("return document.body.innerHTML")
soup = BeautifulSoup(innerHTML,"lxml")

In [81]:
#get who the node is following (leaders)

def get_leaders(soup):
    leaders_list = set()
    leaders = soup.find_all('a', {'class':'link u-baseColor--link avatar u-width60 u-marginRight20 u-flex0'})
    for leader in leaders:
        leader = leader['href']
        leaders_list.add(leader)
    return leaders_list

def get_leaders_dict(soup):
    leaders = soup.find_all('a', {'class':'link u-baseColor--link avatar u-width60 u-marginRight20 u-flex0'})
    i = 0
    leaders_dict = {}
    for leader in leaders:
        i+=1
        leader = leader['href']
        #should put in the peron's url here too
        leaders_dict[i] = [leader,url]
    return leaders_dict

In [82]:
all_the_people.update(get_leaders(soup))
len(all_the_people)

2233

In [83]:
all_the_people

{'https://medium.com/@TGilbertDucree',
 'https://medium.com/@fishspeak',
 'https://medium.com/@melissabrown_15164',
 'https://medium.com/@ChrisSomers1',
 'https://medium.com/@lenambadr',
 'https://medium.com/@mindmedicine',
 'https://medium.com/@bostonwriter',
 'https://medium.com/@asmaaz',
 'https://medium.com/@RyanHigginsUS',
 'https://medium.com/@darnell',
 'https://medium.com/@jjonthan',
 'https://medium.com/@spcolson',
 'https://medium.com/@willemrijksen',
 'https://medium.com/@josephaswanson',
 'https://medium.com/@crazycatkid',
 'https://medium.com/@vshebbal',
 'https://medium.com/@PaulCherryPBR',
 'https://medium.com/@alecberg',
 'https://medium.com/@bootsieju',
 'https://medium.com/@actualblock',
 'https://medium.com/@Ward_Wyatt',
 'https://medium.com/@MeUndies_52932',
 'https://medium.com/@ramblingokie',
 'https://medium.com/@tomjamieson_',
 'https://medium.com/@dawnwildman',
 'https://medium.com/@HoseinPirhady',
 'https://medium.com/@jcbriones',
 'https://medium.com/@jdtarhe

In [84]:
#save the overall list as pickle
import pickle
with open('SenateGOP_all_the_people.pkl', 'wb') as picklefile:
    pickle.dump(all_the_people, picklefile)


In [85]:
leaders = get_leaders_dict(soup)
leaders

{1: ['https://medium.com/@ChandlerHudson', 'https://medium.com/@SenateGOP'],
 2: ['https://medium.com/@dangainor', 'https://medium.com/@SenateGOP'],
 3: ['https://medium.com/@Richard_Zoller', 'https://medium.com/@SenateGOP'],
 4: ['https://medium.com/@kylethewright', 'https://medium.com/@SenateGOP'],
 5: ['https://medium.com/@VAYD', 'https://medium.com/@SenateGOP'],
 6: ['https://medium.com/@tedcruz', 'https://medium.com/@SenateGOP'],
 7: ['https://medium.com/@chuckwestover', 'https://medium.com/@SenateGOP'],
 8: ['https://medium.com/@kippras', 'https://medium.com/@SenateGOP'],
 9: ['https://medium.com/@kavehsaraj', 'https://medium.com/@SenateGOP'],
 10: ['https://medium.com/@stevewymer', 'https://medium.com/@SenateGOP'],
 11: ['https://medium.com/@DanGDaMan', 'https://medium.com/@SenateGOP'],
 12: ['https://medium.com/@g_co', 'https://medium.com/@SenateGOP'],
 13: ['https://medium.com/@mattdole', 'https://medium.com/@SenateGOP'],
 14: ['https://medium.com/@Marks4Maryland', 'https://me

In [86]:
leaders_j = pd.DataFrame(leaders).T
leaders_j.rename(columns = {0:'leader',1:'follower'}, inplace=True)
leaders_j.tail()

,leader,follower
1246,https://medium.com/@alissiahaggard,https://medium.com/@SenateGOP
1247,https://medium.com/@CJArndt,https://medium.com/@SenateGOP
1248,https://medium.com/@ctcreighton,https://medium.com/@SenateGOP
1249,https://medium.com/@genevievemarie_35419,https://medium.com/@SenateGOP
1250,https://medium.com/@lauradonovanua,https://medium.com/@SenateGOP


In [87]:
leaders_j.to_pickle('SenateGOP_leaders.pkl')

# Stop- here for MVP

## combine the leaders and followers dataframes

In [88]:
total_connections = followers_j.append(leaders_j)
total_connections.reset_index(drop=True,inplace=True)
len(total_connections)

2404

In [89]:
total_connections.to_pickle('SenateGOP_connections.pkl')

In [252]:
#test out pickling before closing
# with open("jonathonmorgan_all_the_people.pkl", 'rb') as picklefile: 
#     test = pickle.load(picklefile)
# len(test)

# test = pd.read_pickle('leaders_j.pkl')
# test

## try starting with the list of @johaevans' followers and leaders (from pickle) 

* go to each follower's page
* get their name and the numbeer followers & followees

In [125]:
loaded_leaders = pd.read_pickle('leaders_j.pkl')

In [130]:
loaded_leaders.tail()

,leader,follower
126,https://medium.com/@chelseaclinton,https://medium.com/@johaevans
127,https://medium.com/@StanfordAsh,https://medium.com/@johaevans
128,https://medium.com/@dickc,https://medium.com/@johaevans
129,https://medium.com/@manuel,https://medium.com/@johaevans
130,https://medium.com/@kristoforts,https://medium.com/@johaevans


In [131]:
df = loaded_leaders

In [132]:
nodes = {}

In [133]:
for person in df['leader']:
    #driver = webdriver.Chrome(executable_path="/Users/mayamidzik/metis/2-luther1/selenium/chromedriver")
    url= person
    driver.get(url)
    initialHTML = driver.execute_script("return document.body.innerHTML") #returns the inner HTML as a string
    initialsoup = BeautifulSoup(initialHTML,"lxml")
    
    followers_count = count_followers(initialsoup)
    leaders_count = count_leaders(initialsoup)
    name = get_name(initialsoup)
    
    nodes[url]=({'followers':followers_count})
    nodes[url].update({'leaders':leaders_count})
    nodes[url].update({'name':name})
    
    
    

In [4]:
df_j = pd.DataFrame.from_dict(nodes).T
df_j.reset_index(level=0, inplace=True)
df_j['index'] = df_j
df_j.rename(columns = {'index':'url'},inplace = True)
df_j.head()


NameError: name 'pd' is not defined

In [243]:
df_j.to_pickle('j_leaders_metadata.pkl')

In [186]:
df_j.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 4 columns):
url          130 non-null object
followers    130 non-null object
leaders      130 non-null object
name         130 non-null object
dtypes: object(4)
memory usage: 4.1+ KB


## Pushing data up to RDS

In [7]:
import mysql.connector
import sqlalchemy 
import MySQLdb
import mysql.connector


In [8]:
host="metis.cmwyee1fz5ce.us-east-2.rds.amazonaws.com"
port=3306
dbname="metis"
user="mmidzik"
password="contigo434"

mystring = ('mysql+mysqldb://%s:%s@%s/%s') % (user,password,host,dbname)
myurl = sqlalchemy.engine.url.make_url(mystring) 
engine = sqlalchemy.create_engine(myurl)
con = engine.connect()




In [24]:
df_j = pd.read_pickle('j_leaders_metadata.pkl')
df_j.to_sql(name='new_table_2',con=con,if_exists = 'append', index=False)

/anaconda3/lib/python3.6/site-packages/MySQLdb/cursors.py:318: Warning: (1366, "Incorrect string value: '\\xE9\\x99\\xB3 Jo...' for column 'name' at row 74")
  rows += self.execute(sql + postfix)
/anaconda3/lib/python3.6/site-packages/MySQLdb/cursors.py:318: Warning: (1366, "Incorrect string value: '\\xE2\\x9A\\xA1\\xEF\\xB8\\x8F' for column 'name' at row 107")
  rows += self.execute(sql + postfix)


In [21]:
#df_j.sort_values(by=['name'])
df_j[df_j[['url']='https://medium.com/@WSJ']]

SyntaxError: invalid syntax (<ipython-input-21-98c6d97732bf>, line 2)

In [190]:
df = pd.DataFrame(['A','B'],columns=['new_tablecol'])
df.to_sql(name='new_table3',con=con,if_exists='append')


StatementError: (builtins.AttributeError) 'NoneType' object has no attribute 'cursor' [SQL: 'DESCRIBE `new_table3`']

In [3]:
con.close()